The purpose of this notebook is to experiment with part of speech tagging using a Hidden Markov Model and the Viterbi algoritm. We train the algorithm on the Brown Corpus.

In [ ]:
import numpy as np
import pandas as pd
import nltk

nltk.download('brown')
from nltk.corpus import brown



# Accessing the tagged sentences
nltk.download('universal_tagset')

brown_tagged_sents = brown.tagged_sents(tagset='universal')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


# Brown corpus


In [ ]:
# prompt: From brown_tagged_sents get parts of speech tags and form a set.

parts_of_speech = set()
for sentence in brown_tagged_sents:
  for word, tag in sentence:
    parts_of_speech.add(tag)
print(parts_of_speech)

{'VERB', 'DET', 'PRT', 'ADP', 'ADJ', 'CONJ', 'NUM', 'NOUN', '.', 'PRON', 'X', 'ADV'}


See [universal POS tags readme](https://github.com/slavpetrov/universal-pos-tags/blob/master/README.md)

In [ ]:
brown_tagged_sents = brown.tagged_sents(tagset='universal')
brown_tagged_sents = [[('START', 'START')] + sentence + [('END', 'END')] for sentence in brown_tagged_sents]
# We adding to each sentence a start and end. We can think of 'Start' as a part of speech and also as a word.

In [ ]:
brown_word_tags = [word_tag for sentence in brown_tagged_sents for word_tag in sentence]

In [ ]:
tag_word_pairs = [(tag, word) for word, tag in brown_word_tags]
#reversing the ordering since we want to find prob(word|tag)

In [ ]:
#Getting the conditional frequency distribution for the words which are tagged
cfd_word_given_tag=nltk.ConditionalFreqDist(tag_word_pairs)

In [ ]:
# Create a ConditionalProbDist for emission probabilities
cpd_emission = nltk.ConditionalProbDist(cfd_word_given_tag, nltk.MLEProbDist) #NEED
# LaplaceProbDist is an alternative to MLE

In [ ]:
# Extract the sequence of tags from the original list of (word, tag) pairs
tags = [tag for word, tag in brown_word_tags]

# Create pairs of consecutive tags
tag_pairs = [(tags[i], tags[i+1]) for i in range(len(tags)-1)]


In [ ]:
# Create a CFD from the pairs of consecutive tags
cfd_tag_transitions = nltk.ConditionalFreqDist(tag_pairs) #NEED


In [ ]:
# Convert the CFD into a CPD for transition probabilities
cpd_tag_transitions = nltk.ConditionalProbDist(cfd_tag_transitions, nltk.MLEProbDist) #NEED


# Viterbi Algorithm

In [ ]:
def viterbi(observed_words, cpd_tag_transitions, cpd_emission, states):
    # Initialize the dynamic programming table to store probabilities
    V = [{}]
    path = {}

    # Initialize base case (t == 0)
    for state in states:
        V[0][state] = cpd_tag_transitions['START'].prob(state) * cpd_emission[state].prob(observed_words[0])
        path[state] = [state]

    # Run Viterbi for t > 0
    for t in range(1, len(observed_words)):
        V.append({})
        newpath = {}

        for cur_state in states:
            # Check if state is 'START' or 'END'
            if cur_state in ['START', 'END']:
                continue

            # Select the state transition path with the maximum probability
            # MODIFIED to LOG PROB using ChatGPT
            (prob, state) = max(
                (V[t-1][prev_state] + Math.log(cpd_tag_transitions[prev_state].prob(cur_state)) + Math.log(cpd_emission[cur_state].prob(observed_words[t])), prev_state)
                for prev_state in states if prev_state not in ['START', 'END']
            )

            V[t][cur_state] = prob
            newpath[cur_state] = path[state] + [cur_state]

        # Don't need to remember the old paths
        path = newpath

    # Add a final step for transition to 'END' state
    prob, state = max((V[len(observed_words) - 1][state] * cpd_tag_transitions[state].prob('END'), state) for state in states if state not in ['START', 'END'])
    return (prob, path[state])




In [ ]:
# Example usage
observed_words = ["The", "sly", "fox"]
states = ['NOUN', 'VERB', 'PRON', 'ADJ', 'ADV', 'ADP', 'CONJ', 'DET', 'NUM', 'PRT', 'X', 'START', 'END']  # Add your list of states/tags
(prob, sequence) = viterbi(observed_words, cpd_tag_transitions, cpd_emission, states)
print(f"Probability of the best tag sequence: {prob}")
print(f"Best tag sequence: {sequence}")

Probability of the best tag sequence: 1.1444712915476514e-14
Best tag sequence: ['DET', 'ADJ', 'NOUN']
